In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler


In [ ]:
#匯入數據

file_path = ".\\raw_data.xlsx"
# 讀取資料，跳過前面多餘的標題列，並設置正確的標題行
df = pd.read_excel(file_path)
df


,單位代號,單位名稱,項目,年份,月份,費用
0,A001,工地一號,水費,2024,1,4845
1,A001,工地一號,水費,2024,2,5064
2,A001,工地一號,水費,2024,3,5397
3,A001,工地一號,水費,2024,4,4839
4,A001,工地一號,水費,2024,5,4591
...,...,...,...,...,...,...
175,A005,工地五號,網路費,2024,8,3112
176,A005,工地五號,網路費,2024,9,3124
177,A005,工地五號,網路費,2024,10,3451
178,A005,工地五號,網路費,2024,11,2580


In [ ]:
#根據工地費用數據特性，優化了 Autoencoder 結構與訓練參數，採用較淺層神經網路避免過擬合，並依據不同費用類型動態調整異常判定閾值，提升偵測準確率與實用性。

# 初始化結果儲存
results = []

# 獲取所有單位與項目類型的組合
unit_list = df['單位代號'].unique()
expense_types = df['項目'].unique()

for unit in unit_list:
    for expense in expense_types:
        print(f"正在處理單位：{unit}，項目：{expense}")

        unit_data = df[(df['單位代號'] == unit) & (df['項目'] == expense)].copy()

        if unit_data.shape[0] < 10:
            print(f"⚠️ 單位 {unit} 的 {expense} 數據不足，已跳過")
            continue

        #  標準化費用
        scaler = StandardScaler()
        unit_scaled = scaler.fit_transform(unit_data[['費用']])

        #  定義輸入維度
        input_dim = unit_scaled.shape[1]

        #  建構 Autoencoder 模型
        input_layer = Input(shape=(input_dim,))
        encoded = Dense(4, activation="relu")(input_layer)
        encoded = Dense(2, activation="relu")(encoded)
        decoded = Dense(4, activation="relu")(encoded)
        decoded = Dense(input_dim, activation="linear")(decoded)

        autoencoder = Model(input_layer, decoded)
        autoencoder.compile(optimizer='adam', loss='mse')
        autoencoder.fit(unit_scaled, unit_scaled, epochs=100, batch_size=4, validation_split=0.2, verbose=0)

        #  預測重建誤差
        reconstructed = autoencoder.predict(unit_scaled)
        mse = np.mean(np.power(unit_scaled - reconstructed, 2), axis=1)

        #  設定異常閾值
        mse_threshold = np.percentile(mse, 95)
        unit_data['MSE'] = mse
        unit_data['異常閾值'] = mse_threshold
        unit_data['anomaly'] = mse > mse_threshold

        #  計算平均費用與超出百分比
        avg_fee = unit_data['費用'].mean()
        unit_data['平均費用'] = avg_fee

        unit_data['超出平均百分比'] = np.where(
            avg_fee == 0,
            0,
            ((unit_data['費用'] - avg_fee) / avg_fee) * 100
        )

        results.append(unit_data)

# 合併所有結果
df_result = pd.concat(results)

# 🔹 在這裡插入：匯出完整數據報告
full_report = df_result[['單位代號', '單位名稱', '項目', '年份', '月份', 
                        '費用', '平均費用', '超出平均百分比', 'anomaly']]
full_report.to_excel("費用異常偵測報告.xlsx", index=False)

print("✅ 完整費用異常偵測報告已匯出！（含正常與異常數據）")

# 接著原本的異常明細報告（僅異常）
anomalies = df_result[df_result['anomaly']]
anomalies_report = anomalies[['單位代號', '單位名稱', '項目', '年份', '月份', '費用', '平均費用', '超出平均百分比', 'anomaly']]
anomalies_report.to_excel("異常數據報告.xlsx", index=False)



正在處理單位：A001，項目：水費
1/1 [==============================] - 0s 34ms/step
正在處理單位：A001，項目：電費
1/1 [==============================] - 0s 36ms/step
正在處理單位：A001，項目：網路費
1/1 [==============================] - 0s 35ms/step
正在處理單位：A002，項目：水費
1/1 [==============================] - 0s 35ms/step
正在處理單位：A002，項目：電費
1/1 [==============================] - 0s 36ms/step
正在處理單位：A002，項目：網路費
1/1 [==============================] - 0s 36ms/step
正在處理單位：A003，項目：水費
1/1 [==============================] - 0s 35ms/step
正在處理單位：A003，項目：電費
1/1 [==============================] - 0s 35ms/step
正在處理單位：A003，項目：網路費
1/1 [==============================] - 0s 34ms/step
正在處理單位：A004，項目：水費
1/1 [==============================] - 0s 36ms/step
正在處理單位：A004，項目：電費
1/1 [==============================] - 0s 37ms/step
正在處理單位：A004，項目：網路費
1/1 [==============================] - 0s 35ms/step
正在處理單位：A005，項目：水費
1/1 [==============================] - 0s 36ms/step
正在處理單位：A005，項目：電費
1/1 [==============================] - 0s 37ms/step
正在處理單位：A005，項目：網